In [1]:
import sqlalchemy

from sqlalchemy import create_engine

engine = create_engine('sqlite:///jihun.db', echo = True)
print(engine)


Engine(sqlite:///jihun.db)


In [11]:
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey

metadata = MetaData()
users = Table('users', metadata,
    Column('id', Integer, primary_key=True),
    Column('name', String),
    Column('fullname', String),
)

addresses = Table('addresses', metadata,
    Column('id', Integer, primary_key=True),
    Column('user_id', None, ForeignKey('users.id')),
    Column('email_address', String, nullable=False)
)

In [12]:
insert = users.insert()
print(insert) 
# 커리인지 확인

insert = users.insert().values(name = 'kim', fullname= 'Anonymous, Kim')
print(insert)

insert.compile().params

INSERT INTO users (id, name, fullname) VALUES (:id, :name, :fullname)
INSERT INTO users (name, fullname) VALUES (:name, :fullname)


{'name': 'kim', 'fullname': 'Anonymous, Kim'}

In [13]:
conn = engine.connect()
conn

insert.bind = engine
str(insert)

cur = conn.execute(insert) #result가 프락시(커서 객체)
cur.inserted_primary_key

2019-07-09 13:51:57,945 INFO sqlalchemy.engine.base.Engine INSERT INTO users (name, fullname) VALUES (?, ?)
2019-07-09 13:51:57,947 INFO sqlalchemy.engine.base.Engine ('kim', 'Anonymous, Kim')
2019-07-09 13:51:57,952 INFO sqlalchemy.engine.base.Engine ROLLBACK


OperationalError: (sqlite3.OperationalError) no such table: users [SQL: 'INSERT INTO users (name, fullname) VALUES (?, ?)'] [parameters: ('kim', 'Anonymous, Kim')] (Background on this error at: http://sqlalche.me/e/e3q8)

In [2]:
from sqlalchemy import Table, Column, Integer, String, MetaData, ForeignKey

metadata = MetaData()

In [3]:
artist = Table("Artist", metadata, 
              Column("id", Integer, primary_key=True),
              Column("name", String, nullable=False),
              extend_existing=True)

album = Table("Album", metadata, 
              Column("id", Integer, primary_key=True),
              Column("title", String, nullable=False),
              Column("artist_id", Integer, ForeignKey("Artist.id")),
              extend_existing=True)

genre = Table("Genre", metadata, 
              Column("id", Integer, primary_key=True),
              Column("name", String, nullable=False),
              extend_existing=True)

track = Table("Track", metadata, 
              Column("id", Integer, primary_key=True),
              Column("name", String, nullable=False),
              Column("length", Integer, nullable = False),
              Column("rating", Integer, nullable = False),
              Column("count", Integer, nullable = False),
              Column("album_id",Integer, ForeignKey("Album.id")),
              Column("artist_id", Integer, ForeignKey("Artist.id")),
              extend_existing=True)

In [4]:
metadata.create_all(engine)

2019-07-09 13:40:15,038 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-07-09 13:40:15,040 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 13:40:15,044 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-07-09 13:40:15,047 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 13:40:15,052 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Artist")
2019-07-09 13:40:15,054 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 13:40:15,060 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Album")
2019-07-09 13:40:15,061 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 13:40:15,065 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Genre")
2019-07-09 13:40:15,067 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 13:40:15,070 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Track")
2019-07-09 13:40:15,072 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 13:40:15,077 INFO sqlalchemy.engin

**Show Tables**

In [7]:
tables = metadata.tables
for table in tables:
    print(table)
    
for table in engine.table_names():
    print(table)

Artist
Album
Genre
Track
2019-07-09 13:44:59,446 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2019-07-09 13:44:59,448 INFO sqlalchemy.engine.base.Engine ()
Album
Artist
Genre
Track


In [14]:
conn.execute(artist.insert(), [{"name":"Ji HUn"},{"name":"AC/DC"}])
conn.execute(album.insert(), [{"title":"a","artist_id":1}, {"title":"hungry", "artist_id:":2}])
conn.execute(genre.insert(), [{"name":"Rock"}, {"name":"Metal"}])
conn.execute(track.insert(), [{"title":"dog", "rating":5, "length":111, "count":0, "album_id":1, "genre_id":1},
                             {"title":"cat", "rating":5, "length":222, "count":0, "album_id":1, "genre_id":1},
                             {"title":"apple", "rating":5, "length":443, "count":0, "album_id":2, "genre_id":2},
                             {"title":"brown", "rating":5, "length":131, "count":0, "album_id":2, "genre_id":2}])

2019-07-09 13:52:23,659 INFO sqlalchemy.engine.base.Engine INSERT INTO "Artist" (name) VALUES (?)
2019-07-09 13:52:23,663 INFO sqlalchemy.engine.base.Engine (('Ji HUn',), ('AC/DC',))
2019-07-09 13:52:23,670 INFO sqlalchemy.engine.base.Engine COMMIT
2019-07-09 13:52:23,683 INFO sqlalchemy.engine.base.Engine ROLLBACK


StatementError: (sqlalchemy.exc.InvalidRequestError) A value is required for bind parameter 'artist_id', in parameter group 1 [SQL: 'INSERT INTO "Album" (title, artist_id) VALUES (?, ?)'] [parameters: [{'title': 'a', 'artist_id': 1}, {'title': 'hungry', 'artist_id:': 2}]] (Background on this error at: http://sqlalche.me/e/cd3x)

In [15]:
# execute 되는 순간 인스턴스화해서 메모리에 있는거지 데이터베이스에 있는 것이 아니다
# 엔진이 데이터베이스 물고 있다. 테이블은 데이터베이스에 있는 걸 얘기하는 것

In [16]:
print(track.join(album))

"Track" JOIN "Album" ON "Album".id = "Track".album_id


In [20]:
print(track.join(album).join(genre))
print(track.join(album).join(artist))
print(track.join(album).join(genre).join(artist))

NoForeignKeysError: Can't find any foreign key relationships between 'Join object on Track(685771173504) and Album(685771171656)' and 'Genre'.

In [21]:
conn.close()
metadata.clear()

In [23]:
metadata.tables

immutabledict({})

In [25]:
tables = metadata.tables
for table in tables:
    print(table)
    
for table in engine.table_names():
    print(table)

2019-07-09 14:21:28,581 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2019-07-09 14:21:28,583 INFO sqlalchemy.engine.base.Engine ()
Album
Artist
Genre
Track


In [26]:
metadata = MetaData(bind=engine, reflect=True)
metadata.reflect(bind=engine)

2019-07-09 14:23:28,386 INFO sqlalchemy.engine.base.Engine SELECT name FROM sqlite_master WHERE type='table' ORDER BY name
2019-07-09 14:23:28,388 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 14:23:28,395 INFO sqlalchemy.engine.base.Engine PRAGMA table_info("Album")
2019-07-09 14:23:28,398 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 14:23:28,404 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'Album' AND type = 'table'
2019-07-09 14:23:28,406 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 14:23:28,411 INFO sqlalchemy.engine.base.Engine PRAGMA foreign_key_list("Album")
2019-07-09 14:23:28,414 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 14:23:28,417 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'Album' AND type = 'table'
2019-07-09 14:23:28,419 INFO sqlalchemy.engine.base.Engine ()
2019-07

C:\Users\SAMSUNG\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SADeprecationWarning: reflect=True is deprecate; please use the reflect() method.
  """Entry point for launching an IPython kernel.


2019-07-09 14:23:28,561 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'Track' AND type = 'table'
2019-07-09 14:23:28,563 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 14:23:28,568 INFO sqlalchemy.engine.base.Engine PRAGMA foreign_key_list("Track")
2019-07-09 14:23:28,571 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 14:23:28,575 INFO sqlalchemy.engine.base.Engine SELECT sql FROM  (SELECT * FROM sqlite_master UNION ALL   SELECT * FROM sqlite_temp_master) WHERE name = 'Track' AND type = 'table'
2019-07-09 14:23:28,577 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 14:23:28,584 INFO sqlalchemy.engine.base.Engine PRAGMA index_list("Track")
2019-07-09 14:23:28,586 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 14:23:28,590 INFO sqlalchemy.engine.base.Engine PRAGMA index_list("Track")
2019-07-09 14:23:28,592 INFO sqlalchemy.engine.base.Engine ()
2019-07-09 14:23:28,596 INFO sqlalchemy.engi

In [27]:
for row in metadata.tables:
    print(row)

Album
Artist
Genre
Track
